In [2]:
import glob
# read is2_calval
import geopandas as gpd 
import numpy as np
import os
import glob
import time
import subprocess
import multiprocessing
from tqdm import tqdm
# read H5 file 
import h5py
########
import utm
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.stats import poisson
from shapely.geometry import Point


In [65]:
gdf_is2 = gpd.read_parquet('../result/is2_20m_calval_09252023.parquet') # 18,550,387 IS2 20 m segment points
# Specify the path to your GeoPackage file
als_sites = "../data/sites_20221006.gpkg"
# Read the GeoPackage file
gdf_als = gpd.read_file(als_sites)
# Now, gdf is a GeoDataFrame containing the data from the GeoPackage file

In [223]:
##### get project name 
for index, row in gdf_als[3:4].iterrows(): 
        # Open a text file named "output.txt" in write mode
        # with open('processing.txt', 'w') as file:
        #     # Write text to the file
        #     file.write('# index: {}\n'.format(index))
        #     file.write('# now processing als project: {}\n'.format(row['region'] + '_' + row['name']))
        # The file will be automatically closed when you exit the 'with' block
        # the bounding box of each input geometry intersects the bounding box
        als_index = gdf_is2.sindex.query(row['geometry']) # super fast!!!!! # but only boundary box. 
        is2_in_als = gdf_is2.loc[als_index] 
        #print(len(is2_in_als))
        is2_in_als = is2_in_als.clip(row['geometry'])  # get points inside polygon.
        #out_name = '../result/is2_calval_region/is2_' + row['region'] + '_' + row['name'] + '.parquet'
        print('# now processing als project: ', row['region'] + '_' + row['name'])
        #is2_in_als.to_parquet(out_name)   # save is2 footprints in this als site
        print("# is2 points in this als project: ", len(is2_in_als))
        print('# convert segments in utm...')
        ##############################################################
        for is2_index, is2_row in is2_in_als.iterrows():   
                lat_c = is2_row['land_segments/latitude_20m'] 
                lon_c = is2_row['land_segments/longitude_20m']
                e, n , zone, letter = utm.from_latlon(lat_c, lon_c)
                is2_in_als.loc[is2_index, 'e'] = e
                is2_in_als.loc[is2_index, 'n'] = n
                is2_in_als.loc[is2_index, 'zone'] = zone
        # Create a GeoDataFrame with a Point geometry
        geometry = [Point(x, y) for x, y in zip(is2_in_als['e'], is2_in_als['n'])]
        is2_in_als_utm = gpd.GeoDataFrame(data = is2_in_als, geometry=geometry)
        ##################################################################################
        data_path = '/gpfs/data1/vclgp/data/gedi/imported/' + row['region'] + '/' + row['name'] + '/LAZ_ground'
        # convert all laz file to las files
        files_path = data_path + '/*.laz' 
        laz_files = glob.glob(files_path)
        print('# number of laz files: ', len(laz_files))
        nCPU = len(laz_files)
        if nCPU > 5 : 
           nCPU = 5  # number of cores to use  
        print('# parallel processing...')
        pool = multiprocessing.Pool(nCPU) # Set up multi-processing
        progress_bar = tqdm(total=len(laz_files))
        def update_progress_bar(_):
              progress_bar.update()  
        for laz_f in laz_files:
            pool.apply_async(get_sim_laz, (is2_in_als_utm, laz_f), callback=update_progress_bar)
        pool.close()
        pool.join()
        # Close the progress bar
        progress_bar.close()

# now processing als project:  usa_fernan_tepeepottercreek
# is2 points in this als project:  285
# convert segments in utm...
# number of laz files:  24
# parallel processing...


  0%|                         | 0/24 [00:00<?, ?it/s]

# number of is2 20m segments in las file: # number of is2 20m segments in las file:   148

# number of is2 20m segments in las file:  37
# number of is2 20m segments in las file:  35
# number of is2 20m segments in las file:  25
# number of is2 20m segments in las file:  11


 17%|██▊              | 4/24 [00:00<00:00, 29.15it/s]

# number of is2 20m segments in las file:  52


...

 46%|███████▎        | 11/24 [00:06<00:10,  1.29it/s]

# number of is2 20m segments in las file:  25


 50%|████████        | 12/24 [00:08<00:10,  1.13it/s]

# number of is2 20m segments in las file:  11


 58%|█████████▎      | 14/24 [00:23<00:17,  1.70s/it]


In [222]:
#### 1 segnment ----29 footprints ---find id in the waveform .
# give a laz, create a 20m buffer zone, return all laz in the zone. 
# return all is2 points in laz boundary 
def get_sim_laz(is2_in_als_utm , laz_path):
#laz_path = '/gpfs/data1/vclgp/data/gedi/imported/usa/usda_or/LAZ_ground/or_10_101.laz'
                    als_name = laz_path.split('/')[-3]
                    region = laz_path.split('/')[-4]
                    bounds_file = '/gpfs/data1/vclgp/data/gedi/imported/lists/ground_bounds/boundGround.' + als_name+ '.txt'
                    df = pd.read_csv(bounds_file, header = None, sep = " ")
                    #print('# las file : \n', laz_path)
                    df.columns = ['file', 'xmin', 'ymin', 'zmin', 'xmax', 'ymax', 'zmax']
                    basename = os.path.basename(laz_path)
                    d_file = df[df['file'].str.contains(basename[:-4])]
                    #print( '# laz file bounds: ',d_file.iloc[:, 1:6])
                    xmin = d_file['xmin']
                    xmax = d_file['xmax'] 
                    ymin = d_file['ymin'] 
                    ymax = d_file['ymax'] 
                    # get laz and nearby laz 
                    from shapely.geometry import Polygon
                    # Create a Polygon geometry from the coordinates
                    polygon = Polygon([(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)])
                    # return is2_in_als -----
                    is2_laz = is2_in_als_utm.clip(polygon)
                    #print('# is2_laz: \n', is2_laz)
                    if (len(is2_laz) == 0):
                        #print('# no is2 20m segment is in this las file!')
                        return None
                    # get footprints 
                    ###########################################################
    ###########################################################################
                    # should i get 1 las file  ---> 1 wave.h5?
                    # start from one segment -- one row 
                    print('# number of is2 20m segments in las file: ', len(is2_laz)) 
                    ## ########read simulation waveform file 
                    out_wave = '../wave/wave_'+ basename[:-4] + '.h5'
                    #print('# waveform file: ', out_wave)
                    f_wave = h5py.File(out_wave, 'r')
                    byte_strings = f_wave['WAVEID'][()]
                    wave_ids = []
                    for item in byte_strings:
                        # Convert each byte string to a regular string and join them
                        result_string = ''.join([byte.decode('utf-8') for byte in item])
                        wave_ids.append(result_string)
                    # Find common strings
                    df1 = pd.DataFrame(wave_ids)
                    df1.columns = ['id']
                    df1['wave_index'] = df1.index
                    #print("Keys in HDF5 file:", list(f_wave.keys()))
                    ########## loop every segment 
                    res_las = []
                    for index_20m, row_20m in is2_laz.iterrows():
                                
                                lat_c = row_20m['land_segments/latitude_20m'] 
                                lon_c = row_20m['land_segments/longitude_20m'] 
                                slope = row_20m['slope']
                                #print(out_wave, lat_c, lon_c, slope, '\n')
                                e1, n1 , zone1, letter1 = utm.from_latlon(lat_c, lon_c)
                                theta = math.atan(slope)
                                data = np.arange(-14, 15) # -14 --14
                                e_array = e1 + data * 0.7*math.cos(theta)
                                n_array = n1 + data * 0.7*math.sin(theta)
                                #print('# e_array:', e_array.dtype)
                                e_array = [f'{round(e, 6):.6f}' for e in e_array]
                                n_array = [f'{round(e, 6):.6f}' for e in n_array]  
                                # get unique footprint ID like string. 
                                # Concatenate elements with '.' separator
                                is2_footprintID = [str(e) + '.' + str(n) for e, n in zip(e_array, n_array)]
                                #print('# is2_footprintID', is2_footprintID)
                                ##### I have utm now, return waveid  rowindex start and end. 
                                # get wave id start index
                                df2 = pd.DataFrame(is2_footprintID)
                                df2.columns = ['id']
                                res = pd.merge(df1, df2, on='id', how='inner')
                                #print('res:' , res)
                                start = res['wave_index'].iloc[0]
                                end = res['wave_index'].iloc[-1]
                                #print('start, end: ', start, end)
                                rh = get_sim_rh(out_wave, start, end)
                                rh['fid'] = index_20m
                                res_las.append(rh)
                    f_wave.close()            
                    res_las = pd.concat(res_las, ignore_index=True)
                    out_rh = '../simResult/rh_'+ basename[:-4] + '.parquet'
                    res_las.to_parquet(out_rh)
                    #segment_footprints_utm[['e', 'n']].to_csv(out_coor, sep=' ', header = False,  index = False)
                    #print('# get ch98 , rh and fid ???')
                    # df_tmp = pd.DataFrame()
                    # try:
                    #      df_tmp = get_sim_rh(out_wave)
                    #      df_tmp['fid'] = fid
                    # #      print('# sim result length: ', len(df_tmp))
                    #      results.append(df_tmp)
                    # except:
                    #      pass
                    # if (len(results) > 0):
                    #             # write to parquet 
                    #             las_sim_result = pd.concat(results, ignore_index=True)
                    #             print('# write simulation result for this las file...')
                    #             las_sim_result.to_parquet('../wave/sim_las_'+ basename[:-3] + 'parquet')
                    #             #return pd.concat(results, ignore_index=True)
#laz_path = '/gpfs/data1/vclgp/data/gedi/imported/usa/neon_sawb/LAZ_ground/L26_2014_BURL_1_v01_2014053013_P01_r.laz'                        
#sim_is2_laz(is2_in_als_utm , laz_path)

In [217]:
def get_sim_rh(filename, start, end):
                    # Canopy height, ch
                    # For a 100 m segment, the cumulative height distribution of canopy photons is 
                    # generated and the height at the 98th percentile.
                    with h5py.File(filename, "r") as f:
                        # Print all root level object names (aka keys) 
                        # these can be group or dataset names 
                        #print("Keys: %s" % f.keys())
                        N_foorprints = f['RXWAVECOUNT'].shape[0]
                        pho_w_ground = pd.DataFrame()
                        pho_no_ground = pd.DataFrame()
                        for i in range(start, end+1): # inclu
                                    
                                    RXWAVECOUNT = f['RXWAVECOUNT'][i]
                                    GRWAVECOUNT = f['GRWAVECOUNT'][i]
                                    zStart = f["Z0"][i]
                                    zEnd = f["ZN"][i]
                                    zG = f["ZG"][i]
                                    wfCount = f["NBINS"][0]
                                    if (wfCount < 1): continue
                                    wfStart = 1
                                    # Calculate zStretch
                                    zStretch = zEnd + (np.arange(wfCount, 0, -1) * ((zStart - zEnd) / wfCount))
                                    #plt.plot(RXWAVECOUNT, zStretch,color='red' )
                                    #plt.plot(RXWAVECOUNT - GRWAVECOUNT, zStretch,color='black', linestyle='--' )
                                    #plt.show()
                                    # sampling 
                                    n = np.random.poisson(3,1)[0] # posson sample, how many photons?
                                    
                                    # canopy + ground photons
                                    rows = np.arange(RXWAVECOUNT.shape[0])
                                    total = sum(RXWAVECOUNT)
                                    
                                    # Normalize the data by dividing each value by the total
                                    p_data = [value / total for value in RXWAVECOUNT]
                                    photon_rows = np.random.choice(rows, size=n, p=p_data)
                                    df1 = zStretch[photon_rows] - zG
                                    df1 = pd.DataFrame(df1)
                                    if (len(pho_w_ground) == 0):
                                        pho_w_ground = df1
                                    else:
                                        #print(df1)
                                        pho_w_ground = pd.concat([pho_w_ground, df1], ignore_index=True)
                                    # # canopy photons only
                                    canopy_wave = RXWAVECOUNT - GRWAVECOUNT
                                    
                                        
                                    total = sum(canopy_wave)
                                    if (total == 0 ): continue
                                    # fill NAs.
                                    #canopy_wave = np.nan_to_num(canopy_wave, nan=0.0)
                                    # Normalize the data by dividing each value by the total
                                    p_data = [value / total for value in canopy_wave]
                                    photon_rows = np.random.choice(rows, size=n, p=p_data)
                                    df1 = zStretch[photon_rows] - zG
                                    df1 = pd.DataFrame(df1)
                                    if (len(pho_no_ground) == 0):
                                        pho_no_ground = df1
                                    else:
                                        pho_no_ground = pd.concat([pho_no_ground, df1], ignore_index=True)
                    f.close()
                    percentiles = np.arange(1, 101, 1)
                    height_percentiles = np.percentile(pho_no_ground, percentiles)
                    ch_98 = height_percentiles[97] ### h_98
                    
                    # rh 
                    percentiles = np.arange(0, 101, 1)
                    #0 --> min height
                    # 100 --> max height
                    height_percentiles = np.percentile(pho_w_ground, percentiles)
                    # data frame from 0 to 100
                    column_names = ["rh" + str(i) for i in range(101)]  # Creates a list from rh0 to rh100
                    sim = pd.DataFrame(columns=column_names)
                    # Add your array as a row to the DataFrame
                    sim.loc[0] = height_percentiles
                    sim['h_canopy_98'] = ch_98
                    return sim

In [199]:
for i in range(1,2): print(i)

1


In [224]:
pd.read_parquet('/gpfs/data1/vclgp/xiongl/ProjectIS2CalVal/simResult/rh_Fernan_11.parquet')

,rh0,rh1,rh2,rh3,rh4,rh5,rh6,rh7,rh8,rh9,...,rh93,rh94,rh95,rh96,rh97,rh98,rh99,rh100,h_canopy_98,fid
0,-1.112846,-0.979255,-0.682002,-0.272380,0.074971,0.307222,0.470962,0.578526,0.597129,0.616675,...,26.776775,27.185872,27.510227,27.859304,28.354926,29.165505,30.486582,32.418333,27.934411,14822711
1,-5.196144,-4.954432,-4.655546,-3.657967,-1.966265,-1.270073,-1.161942,-1.153060,-1.081060,-0.853060,...,18.081778,18.116287,18.130810,18.164060,19.118424,20.005204,20.821157,21.919175,24.430088,14822712
2,-2.787182,-2.532182,-2.175873,-1.877455,-1.442455,-0.988518,-0.948283,-0.706667,-0.692455,-0.206374,...,15.708545,15.735792,15.796568,16.302861,16.729275,17.100764,17.822565,18.644054,18.187183,14823351
3,-2.865242,-2.724478,-2.480056,-2.382925,-2.150531,-2.075107,-2.054038,-1.111345,-1.005919,-0.841100,...,12.599616,12.851235,13.717954,14.070896,14.233718,15.570755,15.710014,17.449644,13.029416,14823972
4,-1.788988,-1.555515,-1.322043,-1.169638,-1.098300,-1.026963,-1.026963,-1.026963,-1.022104,-1.012388,...,14.610835,14.743130,14.875424,14.943471,15.011518,15.370693,16.020996,16.671299,14.203593,14824628
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14825255
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14822709
7,-2.481717,-1.592779,-1.404435,-1.323310,-0.988443,-0.838313,-0.824842,-0.787239,-0.753265,-0.627152,...,20.091528,20.429036,21.419010,21.988261,22.231408,22.336672,22.348553,24.999180,25.884180,14822710


In [98]:
# out_wave = '../wave/wave_'+ basename[:-4] + '.h5'
# List of input HDF5 file paths (replace with your file paths)
# no need to merge, process  laz file by laz file. 
data_sim = '../wave/wave_*.h5'
files = glob.glob(data_sim)
print(len(files))
files[100]

9491


'../wave/wave_or_22_137.h5'

In [186]:
# read h5 file , merge all h5 files 
import h5py
# Open the HDF5 file for reading
# waveform file:  ../wave/wave_Fernan_11.h5
file = h5py.File('../wave/wave_Fernan_54.h5', 'r')
# List the keys (datasets and groups) in the HDF5 file
print("Keys in HDF5 file:", list(file.keys()))

Keys in HDF5 file: ['BEAMDENSE', 'FSIGMA', 'GRWAVECOUNT', 'GRWAVEFRAC', 'GRWAVEINT', 'IDLENGTH', 'INCIDENTANGLE', 'LAT0', 'LON0', 'NBINS', 'NPBINS', 'NTYPEWAVES', 'NWAVES', 'POINTDENSE', 'PRES', 'PSIGMA', 'PULSE', 'RXWAVECOUNT', 'RXWAVEFRAC', 'RXWAVEINT', 'SLOPE', 'WAVEID', 'Z0', 'ZG', 'ZGDEM', 'ZN']


In [195]:
byte_strings = file['WAVEID'][()]
wave_ids = []
for item in byte_strings:
        # Convert each byte string to a regular string and join them
        result_string = ''.join([byte.decode('utf-8') for byte in item])
        wave_ids.append(result_string)
# Find common strings
df1 = pd.DataFrame(wave_ids)
df1.columns = ['id']
df1['wave_idex'] = df1.index
df2 = pd.DataFrame(is2_footprintID)
df2.columns = ['id']
res = pd.merge(df1, df2, on='id', how='inner')
start = res['wave_idex'].iloc[0]
end = res['wave_idex'].iloc[-1]
start.dtype

dtype('int64')

In [185]:
# waveform file:  ../wave/wave_Fernan_11.h5
#../wave/wave_Fernan_54.h5  47.779273986816406   -116.37858581542969   -9.696416800718838 
lat_c = 47.779273986816406 
lon_c = -116.37858581542969
slope = -9.696416800718838 
e1, n1 , zone1, letter1 = utm.from_latlon(lat_c, lon_c)
theta = math.atan(slope)
data = np.arange(-14, 15) # -14 --14
e_array = e1 + data * 0.7*math.cos(theta)
n_array = n1 + data * 0.7*math.sin(theta)

e_array = [f'{round(e, 6):.6f}' for e in e_array]
n_array = [f'{round(e, 6):.6f}' for e in n_array]

is2_footprintID = [str(e) + '.' + str(n) for e, n in zip(e_array, n_array)]
is2_footprintID

['546550.996991.5291964.621877',
 '546551.068802.5291963.925570',
 '546551.140613.5291963.229264',
 '546551.212424.5291962.532957',
 '546551.284234.5291961.836650',
 '546551.356045.5291961.140343',
 '546551.427856.5291960.444036',
 '546551.499666.5291959.747729',
 '546551.571477.5291959.051422',
 '546551.643288.5291958.355116',
 '546551.715099.5291957.658809',
 '546551.786909.5291956.962502',
 '546551.858720.5291956.266195',
 '546551.930531.5291955.569888',
 '546552.002342.5291954.873581',
 '546552.074152.5291954.177275',
 '546552.145963.5291953.480968',
 '546552.217774.5291952.784661',
 '546552.289585.5291952.088354',
 '546552.361395.5291951.392047',
 '546552.433206.5291950.695740',
 '546552.505017.5291949.999434',
 '546552.576828.5291949.303127',
 '546552.648638.5291948.606820',
 '546552.720449.5291947.910513',
 '546552.792260.5291947.214206',
 '546552.864070.5291946.517899',
 '546552.935881.5291945.821592',
 '546553.007692.5291945.125286']

In [8]:
import geopandas as gpd
gdf1 = gpd.read_parquet("/gpfs/data1/vclgp/xiongl/ProjectIS2CalVal/data/all_sites_20231218.parquet")

In [9]:
gdf1.head()

,region,name,area_ha,epsg,geometry
0,usa,usda_me,562710.549007,26919,"MULTIPOLYGON (((-70.48145 45.36351, -70.48145 ..."
1,usa,plumasnf_20180707,861751.014503,3310,"MULTIPOLYGON (((-121.56314 39.67990, -121.5631..."
2,australia,tern_robson_whole,3428.721642,28355,"POLYGON ((145.62333 -17.12994, 145.61720 -17.1..."
3,seasia,jpl_borneo_013,3829.727945,32750,"POLYGON ((114.46872 -2.15843, 114.46872 -2.158..."
4,usa,stri_serc_Oct2011,1263.062788,32618,"MULTIPOLYGON (((-76.56098 38.87436, -76.56098 ..."
